# Ungraded Lab: Build a Multi-output Model

In this lab, we'll show how you can build models with more than one output. The dataset we will be working on is available from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency). It is an Energy Efficiency dataset which uses the bulding features (e.g. wall area, roof area) as inputs and has two outputs: Cooling Load and Heating Load. Let's see how we can build a model to train on this data.

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

## Utilities

We define a few utilities for data conversion and visualization to make our code more neat.

In [2]:
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


def plot_diff(y_true, y_pred, title=''):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_' + metric_name], color='green', label='val_' + metric_name)
    plt.show()

## Prepare the Data

We download the dataset and format it for training.

In [3]:
# Specify data URI
URI = './data/ENB2012_data.xlsx'

# Use pandas excel reader
df = pd.read_excel(URI)
df = df.sample(frac=1).reset_index(drop=True)

# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

# Get Y1 and Y2 as the 2 outputs and format them as np arrays
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

# Normalize the training and test data
norm_train_X = norm(train)
norm_test_X = norm(test)

## Build the Model

Here is how we'll build the model using the functional syntax. Notice that we can specify a list of outputs (i.e. `[y1_output, y2_output]`) when we instantiate the `Model()` class.

In [4]:
# Define model layers.
input_layer = Input(shape=(len(train .columns),))
first_dense = Dense(units='128', activation='relu')(input_layer)
second_dense = Dense(units='128', activation='relu')(first_dense)

# Y1 output will be fed directly from the second dense
y1_output = Dense(units='1', name='y1_output')(second_dense)
third_dense = Dense(units='64', activation='relu')(second_dense)

# Y2 output will come via the third dense
y2_output = Dense(units='1', name='y2_output')(third_dense)

# Define the model with the input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output, y2_output])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          1152        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           8256        dense_1[0][0]                    
______________________________________________________________________________________________

## Configure parameters

We specify the optimizer as well as the loss and metrics for each output.

In [5]:
# Specify the optimizer, and compile the model with loss functions for both outputs
optimizer = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

## Train the Model

In [ ]:
# Train the model for 500 epochs
history = model.fit(norm_train_X, train_Y,
                    epochs=500, batch_size=10, validation_data=(norm_test_X, test_Y))

Train on 614 samples, validate on 154 samples
Epoch 1/500
614/614 [==============================] - 1s 2ms/sample - loss: 240.0106 - y1_output_loss: 118.4320 - y2_output_loss: 120.0853 - y1_output_root_mean_squared_error: 10.9236 - y2_output_root_mean_squared_error: 10.9857 - val_loss: 70.9718 - val_y1_output_loss: 21.0678 - val_y2_output_loss: 51.7297 - val_y1_output_root_mean_squared_error: 4.4694 - val_y2_output_root_mean_squared_error: 7.1412
Epoch 2/500
614/614 [==============================] - 0s 248us/sample - loss: 29.4045 - y1_output_loss: 11.3020 - y2_output_loss: 18.1811 - y1_output_root_mean_squared_error: 3.3613 - y2_output_root_mean_squared_error: 4.2551 - val_loss: 61.4197 - val_y1_output_loss: 17.0053 - val_y2_output_loss: 44.5591 - val_y1_output_root_mean_squared_error: 4.0481 - val_y2_output_root_mean_squared_error: 6.7106
Epoch 3/500
614/614 [==============================] - 0s 240us/sample - loss: 26.2671 - y1_output_loss: 10.2887 - y2_output_loss: 15.9137 - y1_o

614/614 [==============================] - 0s 264us/sample - loss: 18.8093 - y1_output_loss: 7.2885 - y2_output_loss: 11.5295 - y1_output_root_mean_squared_error: 2.6953 - y2_output_root_mean_squared_error: 3.3978 - val_loss: 14.7538 - val_y1_output_loss: 6.9441 - val_y2_output_loss: 7.9916 - val_y1_output_root_mean_squared_error: 2.5669 - val_y2_output_root_mean_squared_error: 2.8574
Epoch 22/500
614/614 [==============================] - 0s 300us/sample - loss: 20.5639 - y1_output_loss: 7.5257 - y2_output_loss: 12.9392 - y1_output_root_mean_squared_error: 2.7510 - y2_output_root_mean_squared_error: 3.6050 - val_loss: 17.6072 - val_y1_output_loss: 7.8885 - val_y2_output_loss: 10.1974 - val_y1_output_root_mean_squared_error: 2.6966 - val_y2_output_root_mean_squared_error: 3.2149
Epoch 23/500
614/614 [==============================] - 0s 278us/sample - loss: 19.2694 - y1_output_loss: 7.0036 - y2_output_loss: 12.3409 - y1_output_root_mean_squared_error: 2.6510 - y2_output_root_mean_squar

614/614 [==============================] - 0s 271us/sample - loss: 9.1202 - y1_output_loss: 2.9187 - y2_output_loss: 6.2300 - y1_output_root_mean_squared_error: 1.6908 - y2_output_root_mean_squared_error: 2.5023 - val_loss: 21.0895 - val_y1_output_loss: 8.4555 - val_y2_output_loss: 12.2647 - val_y1_output_root_mean_squared_error: 2.8928 - val_y2_output_root_mean_squared_error: 3.5667
Epoch 42/500
614/614 [==============================] - 0s 287us/sample - loss: 8.0375 - y1_output_loss: 2.7406 - y2_output_loss: 5.2789 - y1_output_root_mean_squared_error: 1.6622 - y2_output_root_mean_squared_error: 2.2967 - val_loss: 22.6557 - val_y1_output_loss: 4.9791 - val_y2_output_loss: 18.6904 - val_y1_output_root_mean_squared_error: 2.1816 - val_y2_output_root_mean_squared_error: 4.2304
Epoch 43/500
614/614 [==============================] - 0s 228us/sample - loss: 10.8511 - y1_output_loss: 3.3256 - y2_output_loss: 7.4524 - y1_output_root_mean_squared_error: 1.8301 - y2_output_root_mean_squared_e

Epoch 62/500
614/614 [==============================] - 0s 293us/sample - loss: 5.5745 - y1_output_loss: 1.6931 - y2_output_loss: 3.8659 - y1_output_root_mean_squared_error: 1.3026 - y2_output_root_mean_squared_error: 1.9692 - val_loss: 5.4236 - val_y1_output_loss: 1.7885 - val_y2_output_loss: 3.7999 - val_y1_output_root_mean_squared_error: 1.2971 - val_y2_output_root_mean_squared_error: 1.9342
Epoch 63/500
614/614 [==============================] - 0s 317us/sample - loss: 5.8433 - y1_output_loss: 1.7127 - y2_output_loss: 4.1020 - y1_output_root_mean_squared_error: 1.3056 - y2_output_root_mean_squared_error: 2.0344 - val_loss: 4.9094 - val_y1_output_loss: 1.6268 - val_y2_output_loss: 3.4657 - val_y1_output_root_mean_squared_error: 1.2204 - val_y2_output_root_mean_squared_error: 1.8493
Epoch 64/500
614/614 [==============================] - 0s 259us/sample - loss: 8.5567 - y1_output_loss: 2.4416 - y2_output_loss: 6.0760 - y1_output_root_mean_squared_error: 1.5684 - y2_output_root_mean_s

Epoch 83/500
614/614 [==============================] - 0s 318us/sample - loss: 3.7861 - y1_output_loss: 0.9765 - y2_output_loss: 2.7946 - y1_output_root_mean_squared_error: 0.9924 - y2_output_root_mean_squared_error: 1.6737 - val_loss: 10.3578 - val_y1_output_loss: 2.2504 - val_y2_output_loss: 7.8284 - val_y1_output_root_mean_squared_error: 1.5027 - val_y2_output_root_mean_squared_error: 2.8460
Epoch 84/500
614/614 [==============================] - 0s 267us/sample - loss: 4.9079 - y1_output_loss: 1.2327 - y2_output_loss: 3.6539 - y1_output_root_mean_squared_error: 1.1133 - y2_output_root_mean_squared_error: 1.9153 - val_loss: 5.2426 - val_y1_output_loss: 1.0485 - val_y2_output_loss: 4.1201 - val_y1_output_root_mean_squared_error: 1.0222 - val_y2_output_root_mean_squared_error: 2.0488
Epoch 85/500
614/614 [==============================] - 0s 269us/sample - loss: 3.2277 - y1_output_loss: 0.8389 - y2_output_loss: 2.3883 - y1_output_root_mean_squared_error: 0.9135 - y2_output_root_mean_

Epoch 104/500
614/614 [==============================] - 0s 336us/sample - loss: 3.7033 - y1_output_loss: 0.7566 - y2_output_loss: 2.9284 - y1_output_root_mean_squared_error: 0.8732 - y2_output_root_mean_squared_error: 1.7149 - val_loss: 8.5337 - val_y1_output_loss: 1.4883 - val_y2_output_loss: 6.8520 - val_y1_output_root_mean_squared_error: 1.2195 - val_y2_output_root_mean_squared_error: 2.6545
Epoch 105/500
614/614 [==============================] - 0s 285us/sample - loss: 3.7263 - y1_output_loss: 1.0147 - y2_output_loss: 2.7556 - y1_output_root_mean_squared_error: 1.0044 - y2_output_root_mean_squared_error: 1.6484 - val_loss: 20.2968 - val_y1_output_loss: 4.9395 - val_y2_output_loss: 15.5911 - val_y1_output_root_mean_squared_error: 2.2482 - val_y2_output_root_mean_squared_error: 3.9042
Epoch 106/500
614/614 [==============================] - 0s 278us/sample - loss: 4.7206 - y1_output_loss: 1.2411 - y2_output_loss: 3.4733 - y1_output_root_mean_squared_error: 1.1092 - y2_output_root_m

614/614 [==============================] - 0s 240us/sample - loss: 2.0474 - y1_output_loss: 0.4563 - y2_output_loss: 1.5730 - y1_output_root_mean_squared_error: 0.6782 - y2_output_root_mean_squared_error: 1.2599 - val_loss: 3.7426 - val_y1_output_loss: 0.5228 - val_y2_output_loss: 3.2125 - val_y1_output_root_mean_squared_error: 0.7188 - val_y2_output_root_mean_squared_error: 1.7961
Epoch 125/500
614/614 [==============================] - 0s 249us/sample - loss: 2.3783 - y1_output_loss: 0.4500 - y2_output_loss: 1.9641 - y1_output_root_mean_squared_error: 0.6733 - y2_output_root_mean_squared_error: 1.3874 - val_loss: 6.6179 - val_y1_output_loss: 0.5516 - val_y2_output_loss: 6.5729 - val_y1_output_root_mean_squared_error: 0.7406 - val_y2_output_root_mean_squared_error: 2.4636
Epoch 126/500
614/614 [==============================] - 0s 216us/sample - loss: 1.9181 - y1_output_loss: 0.4160 - y2_output_loss: 1.5014 - y1_output_root_mean_squared_error: 0.6397 - y2_output_root_mean_squared_erro

Epoch 145/500
614/614 [==============================] - 0s 224us/sample - loss: 2.1031 - y1_output_loss: 0.5084 - y2_output_loss: 1.5910 - y1_output_root_mean_squared_error: 0.7095 - y2_output_root_mean_squared_error: 1.2648 - val_loss: 3.4526 - val_y1_output_loss: 0.7977 - val_y2_output_loss: 2.6500 - val_y1_output_root_mean_squared_error: 0.8927 - val_y2_output_root_mean_squared_error: 1.6296
Epoch 146/500
614/614 [==============================] - 0s 220us/sample - loss: 1.6159 - y1_output_loss: 0.3444 - y2_output_loss: 1.2601 - y1_output_root_mean_squared_error: 0.5893 - y2_output_root_mean_squared_error: 1.1263 - val_loss: 2.2679 - val_y1_output_loss: 0.3806 - val_y2_output_loss: 1.9951 - val_y1_output_root_mean_squared_error: 0.6011 - val_y2_output_root_mean_squared_error: 1.3808
Epoch 147/500
614/614 [==============================] - 0s 238us/sample - loss: 1.6375 - y1_output_loss: 0.3534 - y2_output_loss: 1.2847 - y1_output_root_mean_squared_error: 0.5964 - y2_output_root_mea

614/614 [==============================] - 0s 232us/sample - loss: 1.9801 - y1_output_loss: 0.3316 - y2_output_loss: 1.6408 - y1_output_root_mean_squared_error: 0.5753 - y2_output_root_mean_squared_error: 1.2842 - val_loss: 2.8382 - val_y1_output_loss: 0.4674 - val_y2_output_loss: 2.5823 - val_y1_output_root_mean_squared_error: 0.6867 - val_y2_output_root_mean_squared_error: 1.5384
Epoch 166/500
614/614 [==============================] - 0s 244us/sample - loss: 1.6303 - y1_output_loss: 0.3291 - y2_output_loss: 1.3151 - y1_output_root_mean_squared_error: 0.5757 - y2_output_root_mean_squared_error: 1.1397 - val_loss: 5.1547 - val_y1_output_loss: 0.6163 - val_y2_output_loss: 5.0230 - val_y1_output_root_mean_squared_error: 0.7724 - val_y2_output_root_mean_squared_error: 2.1350
Epoch 167/500
614/614 [==============================] - 0s 222us/sample - loss: 1.2652 - y1_output_loss: 0.2609 - y2_output_loss: 0.9947 - y1_output_root_mean_squared_error: 0.5129 - y2_output_root_mean_squared_erro

## Evaluate the Model and Plot Metrics

In [ ]:
# Test the model and print loss and mse for both outputs
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

In [ ]:
# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], title='Y1')
plot_diff(test_Y[1], Y_pred[1], title='Y2')
plot_metrics(metric_name='y1_output_root_mean_squared_error', title='Y1 RMSE', ylim=6)
plot_metrics(metric_name='y2_output_root_mean_squared_error', title='Y2 RMSE', ylim=7)